In [1]:
import logging
from typing import Any, Set

from automata.config.base import AgentConfigName
from automata.core.singletons.dependency_factory import dependency_factory
from automata.core.singletons.py_module_loader import py_module_loader
from automata.core.context_providers.symbol_synchronization import (
    SymbolProviderSynchronizationContext,
)
    
from automata.core.tools.base import Tool
from automata.core.tools.builders.context_oracle import ContextOracleToolkitBuilder

logger = logging.getLogger(__name__)

py_module_loader.initialize()

In [2]:
symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')
symbol_doc_embedding_handler = dependency_factory.get('symbol_doc_embedding_handler')
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')

In [3]:
context_oracle = ContextOracleToolkitBuilder(symbol_search, symbol_doc_embedding_handler, symbol_code_embedding_handler, embedding_similarity_calculator)

In [4]:
context = context_oracle._get_context("Provide new code which refactors the SymbolDocEmbeddingHandler class to be more robust.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 814/814 [00:04<00:00, 164.51it/s]

result =  class SymbolDocEmbeddingHandler(SymbolEmbeddingHandler):
    """A class to handle the embedding of symbols"""

    def __init__(
        self,
        embedding_db: JSONSymbolEmbeddingVectorDatabase,
        embedding_builder: SymbolDocEmbeddingBuilder,
    ) -> None:
        super().__init__(embedding_db, embedding_builder)

    def get_embedding(self, symbol: Symbol) -> SymbolDocEmbedding:
        return self.embedding_db.get(symbol.dotpath)

    def process_embedding(self, symbol: Symbol) -> None:
        """
        Process the embedding for a `Symbol` -
        Currently we do nothing if the symbol is already contained
        """

        source_code = self.embedding_builder.fetch_embedding_source_code(symbol)

        if not source_code:
            raise ValueError(f"Symbol {symbol} has no source code")

        if self.embedding_db.contains(symbol.dotpath):
            self.update_existing_embedding(source_code, symbol)
            return
        # else:
        symb

In [5]:
print(context)

class SymbolDocEmbeddingHandler(SymbolEmbeddingHandler):
    """A class to handle the embedding of symbols"""

    def __init__(
        self,
        embedding_db: JSONSymbolEmbeddingVectorDatabase,
        embedding_builder: SymbolDocEmbeddingBuilder,
    ) -> None:
        super().__init__(embedding_db, embedding_builder)

    def get_embedding(self, symbol: Symbol) -> SymbolDocEmbedding:
        return self.embedding_db.get(symbol.dotpath)

    def process_embedding(self, symbol: Symbol) -> None:
        """
        Process the embedding for a `Symbol` -
        Currently we do nothing if the symbol is already contained
        """

        source_code = self.embedding_builder.fetch_embedding_source_code(symbol)

        if not source_code:
            raise ValueError(f"Symbol {symbol} has no source code")

        if self.embedding_db.contains(symbol.dotpath):
            self.update_existing_embedding(source_code, symbol)
            return
        # else:
        symbol_embeddi